# Model With GPU (Fine tuning the model using LoRA)

In [14]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, PeftConfig
from tqdm import tqdm

## Import the data from Hugging face

In [15]:
# dataset = load_dataset('samhog/psychology-10k', split = ['train'])[0]
# dataset.save_to_disk('./dataset')

dataset = load_from_disk('./dataset')
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9846
})

let's walk trough the data

In [3]:
def print_conv(me, model):
    for i, j in zip(me, model):
        print(
            f"""
Me : {i}

Param mitr : {j}\n\n
            """
        )

In [4]:
print_conv(dataset['input'][:3], dataset['output'][:3])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : It's common to feel anxious at times, and there can be many reasons for it. Have there been any recent changes or stressors in your life that may be contributing to your anxiety? Let's work together to identify any triggers and develop coping strategies to manage your anxiety.


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : It's understandable to feel worried and suspicious in this situation. Have you talked to your partner about your concerns? It's important to communicate openly and honestly with them. If you're still feeling uncertain, we can work on developing a plan to address the situation in a healthy and constructive way.


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : It sounds like you're going through a difficult time. Let's work on creating a 

In [5]:
dataset.shape

(9846, 3)

In [6]:
Dataset = dataset.train_test_split(test_size = 0.2)
Dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 7876
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1970
    })
})

## Import our model with its tokenizer

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

In [17]:
# Importing model from the Hugging Face hub

model_name = 'google/flan-t5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype = torch.bfloat16,
    # quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Saving the model

# model.save_pretrained('/kaggle/working/Model')
# tokenizer.save_pretrained('/kaggle/working/Model')

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
import gc
def clean():
    for i in range(15):
      gc.collect()
      torch.cuda.empty_cache()

In [7]:
# Importing model saved locally
# tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Model', device_map="auto")
# model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
#                                               device_map="auto",
#                                               torch_dtype = torch.bfloat16,
#                                               # quantization_config = bnb_config
#                                              )

Let's try some stuff with tokenizer

In [ ]:
tokenizer("hey", return_token_type_ids = False)

In [ ]:
device = torch.device('cuda')
tokenizer.decode(
    model.generate(
        tokenizer(
            dataset['input'][0], return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device)
    )[0], skip_special_tokens= True
)

In [ ]:
def inference(input_data, param_mitr = model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent + task for sent in input_data]
    output = param_mitr.generate(
        tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device),
        generation_config = GenerationConfig(max_new_token = 200)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [ ]:
inference(dataset['input'][:4])

The model is showing good result on ICL (In Context Learning) with zero shot inference.

In [ ]:
def tokenize_function(example):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + question + '\n' + task for question in example['input']]
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['output'], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


tokenized_datasets = Dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input', 'output', 'instruction'])

## Fine tune our model

In [ ]:
config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r = 32,
    target_modules = "all-linear",
    lora_alpha=32,
    lora_dropout=0.05
)

In [ ]:
Model = get_peft_model(model, config)
Model.print_trainable_parameters()

I have already saved the model and will not retrieve it from the hub

In [ ]:
TrainArgs = TrainingArguments(
    output_dir = './Model',
    learning_rate = 1e-3,
    num_train_epochs = 2,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps=1,
    load_best_model_at_end = True,
    report_to = ['tensorboard']
)

In [ ]:
trainer = Trainer(
    model = Model,
    args = TrainArgs,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

In [ ]:
clean()
trainer.train()

In [26]:
def peft_infrence_with_GPU(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ).to('cuda'),
        generation_config = GenerationConfig(max_new_tokens=100, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [ ]:
peft_infrence_withGPU(dataset['input'][-4:])

In [ ]:
trainer.model.save_pretrained('./peft_for_param_mitr')

# Model without GPU (using saved fine-tuned model)

In [27]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Model', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
                                              device_map="auto",
                                              torch_dtype = torch.bfloat16,
                                              # quantization_config = bnb_config
                                             )

In [28]:
peft_model_path = './peft_for_param_mitr'
Model = PeftModel.from_pretrained(
            model,
            peft_model_path,
            is_trainable = False
            )

In [39]:
def peft_infrence(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ),
        generation_config = GenerationConfig(max_new_tokens=200, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [40]:
peft_infrence(Dataset['test']['input'][:4])


Me : I'm having trouble forgiving someone. How can I move on?

Param mitr : Forgiving someone can be a difficult process, but it's important to move on. Let's work on identifying the root of your forgiving and developing coping mechanisms to help you move on.


            

Me : I'm having trouble with panic attacks and I don't know how to stop them.

Param mitr : Panic attacks can be a difficult and complex issue to manage. Let's work together to identify the triggers and develop coping strategies to manage panic attacks. We can also explore any underlying issues that may be contributing to your panic attacks.


            

Me : I'm having trouble connecting with others and feel lonely. What can I do?

Param mitr : Connect with others can be challenging, but it's important to find ways to connect with others. One approach is to seek support from friends or a therapist. Additionally, seeking support from friends or a therapist can help you connect with others. Additionally, seeking